Ranking system for Martensitic Style Alloys

In [7]:
# Imports
import os
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [8]:
# Composition grids (wt %)
C_values   = [0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 1.00]
Cr_values  = [11.0, 12.0, 13.0, 14.0, 15.0]
X_values   = [0.0, 0.5, 1.0, 1.5, 2.0, 2.5]
N_values   = [0.00, 0.05, 0.10, 0.15]

In [9]:
# Input data 
X_elements = ["Mo", "Ni", "Mn", "Si"]

service_temperature_C = 20

""" https://en.wikipedia.org/wiki/Prices_of_chemical_elements """
cost_usd_per_kg = {
    "Fe": 0.42,
    "C":  0.13,
    "Cr": 9.4,
    "Mo": 40.1,
    "Ni": 13.9,
    "Mn": 1.8,
    "Si": 1.7,
    "N":  0.15,
}

density_proxy = 7.8

In [10]:
# Ranking calculation proxies
def Ms_andrews(C_wt, Cr_wt, Ni_wt=0.0, Mn_wt=0.0, Mo_wt=0.0, N_wt=0.0, include_N_term=False):
    """Andrews-type Ms equation (°C):"""
    Ms = 539 - 423*C_wt - 30.4*Mn_wt - 17.7*Ni_wt - 12.1*Cr_wt - 7.5*Mo_wt
    return Ms

def pren(Cr_wt, Mo_wt=0.0, N_wt=0.0, W_wt=0.0):
    """PREN ≈ %Cr + 3.3(%Mo + 0.5%W) + 16%N  (Si does not enter PREN)"""
    pren = Cr_wt + 3.3*(Mo_wt + 0.5*W_wt) + 16.0*N_wt
    return pren

def strength_proxy(alloy):
    """Solid solution strengthening proxy (MPa)"""
    k = {"Cr": 35.0, "Mo": 120.0, "Ni": 20.0, "Mn": 25.0, "Si": 20.0, "C": 250.0, "N": 180.0}
    s = 0.0
    for el, wt in alloy.items():
        if el in k and wt > 0:
            s += k[el] * math.sqrt(wt)
    return s

In [11]:
# Generate data
rows = []
for X in X_elements:
    for C in C_values:
        for Cr in Cr_values:
            for N in N_values:
                for x_wt in X_values:
                    comp = {"C": C, "Cr": Cr, "N": N, X: x_wt}
                    Ms = Ms_andrews(
                        C_wt=C, Cr_wt=Cr,
                        Ni_wt=comp.get("Ni", 0.0),
                        Mn_wt=comp.get("Mn", 0.0),
                        Mo_wt=comp.get("Mo", 0.0),
                        N_wt=N,
                        include_N_term=False  # classic Andrews form
                    )
                    PREN = pren(Cr_wt=Cr, Mo_wt=comp.get("Mo", 0.0), N_wt=N)
                    dss  = strength_proxy(comp)

                    Fe_balance = 100.0 - (C + Cr + N + x_wt)

                    rows.append({
                        "X_element": X,
                        "C_wt%": C,
                        "Cr_wt%": Cr,
                        "N_wt%": N,
                        f"{X}_wt%": x_wt,
                        "Fe_wt% (balance)": Fe_balance,
                        "Ms_C": Ms,
                        "PREN": PREN,
                        "Strength_proxy_MPa": dss,
                        "Service_T_C (info)": service_temperature_C,
                        "Ms_ok_(>=200C)": Ms >= 200.0,
                        "Fe_balance_ok_(>80%)": Fe_balance > 80.0
                    })

df = pd.DataFrame(rows)

csv_path = "Martensite_Screening.csv"
df.to_csv(csv_path, index=False)
print(f"CSV saved: {csv_path}")
df.head(-10)

CSV saved: Martensite_Screening.csv


,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Fe_wt% (balance),Ms_C,PREN,Strength_proxy_MPa,Service_T_C (info),Ms_ok_(>=200C),Fe_balance_ok_(>80%),Ni_wt%,Mn_wt%,Si_wt%
0,Mo,0.1,11.0,0.00,0.0,88.90,363.60,11.00,195.138809,20,True,True,NaN,NaN,NaN
1,Mo,0.1,11.0,0.00,0.5,88.40,359.85,12.65,279.991623,20,True,True,NaN,NaN,NaN
2,Mo,0.1,11.0,0.00,1.0,87.90,356.10,14.30,315.138809,20,True,True,NaN,NaN,NaN
3,Mo,0.1,11.0,0.00,1.5,87.40,352.35,15.95,342.108194,20,True,True,NaN,NaN,NaN
4,Mo,0.1,11.0,0.00,2.0,86.90,348.60,17.60,364.844437,20,True,True,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4785,Si,1.0,15.0,0.05,NaN,82.45,-65.50,15.80,450.298538,20,False,True,NaN,NaN,1.5
4786,Si,1.0,15.0,0.05,NaN,81.95,-65.50,15.80,454.087912,20,False,True,NaN,NaN,2.0
4787,Si,1.0,15.0,0.05,NaN,81.45,-65.50,15.80,457.426417,20,False,True,NaN,NaN,2.5
4788,Si,1.0,15.0,0.10,NaN,83.90,-65.50,16.60,442.475415,20,False,True,NaN,NaN,0.0


In [ ]:
df_valid = df[(df["Ms_ok_(>=200C)"] == True) & (df["Fe_balance_ok_(>80%)"] == True)]

df_valid["Score"] = df_valid["PREN"] + df_valid["Strength_proxy_MPa"] / 100.0
df_sorted = df_valid.sort_values(by="Score", ascending=False)
df_sorted.head(20)

/var/folders/r_/mmq2gnpx6qnbz26x3ypfrmvh0000gn/T/ipykernel_96097/1480101182.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_valid["Score"] = df_valid["PREN"] + df_valid["Strength_proxy_MPa"] / 100.0


,X_element,C_wt%,Cr_wt%,N_wt%,Mo_wt%,Fe_wt% (balance),Ms_C,PREN,Strength_proxy_MPa,Service_T_C (info),Ms_ok_(>=200C),Fe_balance_ok_(>80%),Ni_wt%,Mn_wt%,Si_wt%,Score
359,Mo,0.3,15.0,0.15,2.5,82.05,211.85,25.65,531.935416,20,True,True,NaN,NaN,NaN,30.969354
239,Mo,0.2,15.0,0.15,2.5,82.15,254.15,25.65,506.808176,20,True,True,NaN,NaN,NaN,30.718082
119,Mo,0.1,15.0,0.15,2.5,82.25,296.45,25.65,474.061718,20,True,True,NaN,NaN,NaN,30.390617
353,Mo,0.3,15.0,0.10,2.5,82.10,211.85,24.85,519.142714,20,True,True,NaN,NaN,NaN,30.041427
335,Mo,0.3,14.0,0.15,2.5,83.05,223.95,24.65,527.339008,20,True,True,NaN,NaN,NaN,29.923390
233,Mo,0.2,15.0,0.10,2.5,82.20,254.15,24.85,494.015473,20,True,True,NaN,NaN,NaN,29.790155
215,Mo,0.2,14.0,0.15,2.5,83.15,266.25,24.65,502.211767,20,True,True,NaN,NaN,NaN,29.672118
113,Mo,0.1,15.0,0.10,2.5,82.30,296.45,24.85,461.269016,20,True,True,NaN,NaN,NaN,29.462690
95,Mo,0.1,14.0,0.15,2.5,83.25,308.55,24.65,469.465310,20,True,True,NaN,NaN,NaN,29.344653
358,Mo,0.3,15.0,0.15,2.0,82.55,215.60,24.00,511.904384,20,True,True,NaN,NaN,NaN,29.119044
